In [1]:
import pandas as pd
import numpy as np
attacks = pd.read_csv(r'attacks.csv', encoding='latin-1')

In [2]:
import re

In [3]:
from pandas_profiling import ProfileReport

In [4]:
profile = ProfileReport(attacks)

In [5]:
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Tratando os dados

In [6]:
sum(attacks['Unnamed: 23'].isna()) #O codigo mostra que ambos só tem valores nulos (foi rodada de 1 a 1)
sum(attacks['Unnamed: 23'].isna())

25721

In [7]:
attacks.drop(['Unnamed: 22','Unnamed: 23','Time','Investigator or Source','pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order'], axis=1, inplace=True) 
#Removendo colunas desnecessarias para a Analise

In [8]:
mask_date_na = attacks['Date'].notna()
attacks = attacks[mask_date_na] #remover linhas em que a data é nula, que já foi constatado que não contem dado
mask_date_1900 = attacks['Year'] >= 1900
attacks = attacks[mask_date_1900] #remover entradas que não foram acima do ano 1900, pois abaixo de 1900 não é relevante para analise
attacks.dropna(subset='Case Number', inplace = True) #removendo nulo de coluna de analise

In [9]:
attacks['Case Number'] = attacks['Case Number'].str[:10] #Removendo letras acima de 10 caracteres

In [10]:
attacks = attacks[~attacks['Case Number'].str.contains('.00')] #Removendo linhas que data está incompleta

In [11]:
attacks['Case Number'] = attacks['Case Number'].str.replace('[-,]','.',regex=True)
#Removendo os caracteres indesejados para formatar para datetime

In [12]:
attacks['Case Number'] = pd.to_datetime(attacks['Case Number'], format="%Y/%m/%d")
#transformando em datetime

In [13]:
mask_unpro = attacks['Type'] == 'Unprovoked'
mask_prov = attacks['Type'] == 'Provoked'
attacks[attacks['Type']!= 'Unprovoked']

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species
0,2018-06-25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark
2,2018-06-09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,NaN
4,2018-06-04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m"
10,2018-05-24,24-May-2018,2018.0,Provoked,AUSTRALIA,Queensland,Cairns Aquarium,Feeding sharks,male,M,32,Minor bite to hand by captive shark. PROVOKED ...,N,Grey reef shark
13,2018-05-13,13-May-2018,2018.0,Invalid,ENGLAND,Cornwall,Off Land's End,Fishing,Max Berryman,M,21,Injured by teeth of a dead porbeagle shark he ...,N,Invalid incident
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5534,1902-07-06,06-Jul-1902,1902.0,Provoked,USA,New Jersey,"Atlantic City, Atlantic County",Swimming,Harry M. Speerman,M,NaN,Shark bit his left arm after he grabbed its ta...,N,NaN
5538,1902-02-22,.Reported 22-Feb-1902,1902.0,Provoked,NEW ZEALAND,North Island,Poverty Bay,Standing on ship deck,male,M,NaN,"No injury, hooked shark bit tousers. PROVOKED ...",N,NaN
5540,1902-01-19,Reported 19-Jan-1902,1902.0,Invalid,SOUTH AFRICA,KwaZulu-Natal,Durban,NaN,a soldier from the Natal Garrison,M,NaN,Possible drowning & scavenging. Remains were r...,NaN,Questionable
5546,1901-07-17,17-Jul-1901,1901.0,Invalid,ITALY,Syracuse,Capo Santa Croce,Swimming,Antonio Tornatori,M,20,"Disappeared, but shark involvement unconfirmed",NaN,Questionable


# Preparação da Analise e Analise

#Pergunta : Qual é a frequencia de ataques de tubarão não provocados e provocados de 1900 até hoje por país e por sexo. 
#Brasil especificamente estados